<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터를 dataframe으로 불러들이고 자료구조를 본다.
이 때 pandas 라이브러리를 사용한다.

In [ ]:
import pandas as pd

# Load the data from the uploaded CSV file
file_path = '/content/NVIDIA_minute.csv'
nvidia_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
nvidia_data.head()


 nvidia_data라는 데이터프레임이 만들어졌다.

In [ ]:
# Combining date and time into a single datetime column and converting it to datetime type
nvidia_data['Datetime'] = pd.to_datetime(nvidia_data['Date'] + ' ' + nvidia_data['Time'])

# Selecting relevant columns and sorting by datetime
nvidia_data = nvidia_data[['Datetime', 'Open', 'High', 'Low', 'Close', 'upvolume', 'downvolume']]
nvidia_data.sort_values('Datetime', inplace=True)

# Handling missing values by filling them with zeros (could consider other imputation techniques depending on the analysis)
nvidia_data.fillna(0, inplace=True)

# Display the processed data to verify changes
nvidia_data.head()


## 시퀀스 데이터 만들기

In [ ]:
import numpy as np

# Initialize the MinMaxScaler
!pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Selecting numerical features for scaling
features = nvidia_data[['Open', 'High', 'Low', 'Close', 'upvolume', 'downvolume']]
scaled_features = scaler.fit_transform(features)

# Function to create sequences for the LSTM model
def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length, 3])  # Index 3 is the 'Close' price
    return np.array(X), np.array(y)

# Creating sequences with a sequence length of 60 minutes
X, y = create_sequences(scaled_features)

# Display shapes of the arrays to verify
X.shape, y.shape

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Selecting numerical features for scaling
features = nvidia_data[['Open', 'High', 'Low', 'Close', 'upvolume', 'downvolume']]
scaled_features = scaler.fit_transform(features)

# Function to create sequences for the LSTM model
def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length, 3])  # Index 3 is the 'Close' price
    return np.array(X), np.array(y)

# Creating sequences with a sequence length of 60 minutes
X, y = create_sequences(scaled_features)

# Display shapes of the arrays to verify
X.shape, y.shape


In [ ]:
!pip install torch torchvision
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# ... Rest of the code remains unchanged ...

# Creating datasets and dataloaders for training
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Defining the LSTM model using PyTorch
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=6, hidden_size=50, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(50, 50, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.dense = nn.Linear(50, 1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x = x[:, -1, :]  # Take the output of the last sequence
        return self.dense(x)

# Initialize the model, loss function, and optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 50
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# The model is now trained.
"Training complete!"
